In [5]:
import pandas as pd

# load dataset
url = "https://raw.githubusercontent.com/fivethirtyeight/data/master/candy-power-ranking/candy-data.csv"
df = pd.read_csv(url)

# saved data also in folder to look at

# show data info
print(df.head())
print(df.info())

  competitorname  chocolate  fruity  caramel  peanutyalmondy  nougat  \
0      100 Grand          1       0        1               0       0   
1   3 Musketeers          1       0        0               0       1   
2       One dime          0       0        0               0       0   
3    One quarter          0       0        0               0       0   
4      Air Heads          0       1        0               0       0   

   crispedricewafer  hard  bar  pluribus  sugarpercent  pricepercent  \
0                 1     0    1         0         0.732         0.860   
1                 0     0    1         0         0.604         0.511   
2                 0     0    0         0         0.011         0.116   
3                 0     0    0         0         0.011         0.511   
4                 0     0    0         0         0.906         0.511   

   winpercent  
0   66.971725  
1   67.602936  
2   32.261086  
3   46.116505  
4   52.341465  
<class 'pandas.core.frame.DataFrame'>


In [2]:
# Korrelationsmatrix berechnen
correlation_matrix = df.corr()

# Korrelationen mit der Beliebtheitsspalte sortieren
popularities = correlation_matrix['winpercent'].sort_values(ascending=False)

print(popularities)

ValueError: could not convert string to float: '100 Grand'